In [53]:
# 라이브러리 임포트
import pandas as pd
import numpy as np

In [54]:
# 디렉토리에 해당 파일 없으면 디렉토리 변경
# import os
# data_directory = '/Users/hamhyewon/Desktop/aepython/찐감자와 공주들'
# os.chdir(data_directory)
# os.getcwd()

In [65]:
# 선수 파일 불러오기
name = '박석민'
player_file_path = f'./merged_data_{name}.xlsx'

# 데이터 읽기
df = pd.read_excel(player_file_path)

# 경기력 일관성 칼럼 추가하기
df['경기력 일관성']=df['2타']+df['3타']+df['홈런']

# 지역별 경기력 일관성 데이터 추출하기
df_Changwon = df[(df['지역'] == '창원') & df['경기력 일관성']]


In [66]:
# 1. 날씨(기온, 풍속, 이슬점) 데이터의 x축 범위를 5구간으로 나누는 사용자 정의 함수 만들기
# 1-1) 범위: (각 데이터별 최고값 - 각 데이터별 최저값) / 5의 값만큼 각 범위에 더해주기
# 1-2) 반올림: 범위별 나온 값을 소수점 셋째 자리까지 반올림한 범위를 만들기
def ranges_and_labels(data, weather_column):
    max_val, min_val = max(data[weather_column]), min(data[weather_column])
    width = (max_val - min_val) / 5
    ranges = [(min_val + num * width, min_val + (num + 1) * width) for num in range(5)]
    rounded_ranges = [(round(start, 3), round(end, 3)) for start, end in ranges]
    labels_x = [f"{start} - {end}" for start, end in rounded_ranges]
    return ranges, labels_x

# 1-3) x_range_weather_val에 x 축 범위 구간 저장
range_temp, x_range_temp = ranges_and_labels(df_Changwon, '평균기온(°C)')
range_wind, x_range_wind = ranges_and_labels(df_Changwon, '평균 풍속(m/s)')
range_humid, x_range_humid = ranges_and_labels(df_Changwon, '평균 이슬점온도(°C)')

print(x_range_temp)
print(x_range_wind)
print(x_range_humid)

['9.0 - 12.94', '12.94 - 16.88', '16.88 - 20.82', '20.82 - 24.76', '24.76 - 28.7']
['0.8 - 1.22', '1.22 - 1.64', '1.64 - 2.06', '2.06 - 2.48', '2.48 - 2.9']
['-9.8 - -3.0', '-3.0 - 3.8', '3.8 - 10.6', '10.6 - 17.4', '17.4 - 24.2']


In [67]:
# 2. 날씨(기온, 풍속, 이슬점) 데이터의 y축 범위를 5구간으로 나누는 사용자 정의 함수 만들기
# 2-1) 인덱스 초기화해 for문 돌아갈 때 인덱스 순서대로 조회될 수 있게 함
# 2-2) 초기화된 리스트에 y 범위별 값을 for문을 돌면서 부여하기 
df_Changwon_reset_idx=df_Changwon.reset_index(drop=True)

def weather_and_hits(data, weather_column, total_hits_column):
    labels_y = [0] * 5
    max_val, min_val = max(data[weather_column]), min(data[weather_column])
    width = (max_val - min_val) / 5

    for date in range(len(data['날짜'])):
        temp = data[weather_column][date]
        for num in range(5):
            min_range = min_val + num * width
            max_range = min_val + (num + 1) * width
            labels_y[num] += data[total_hits_column][date] if min_range < temp <= max_range else 0

    return labels_y

y_range_temp = weather_and_hits(df_Changwon_reset_idx, '평균기온(°C)', '경기력 일관성')
y_range_wind = weather_and_hits(df_Changwon_reset_idx, '평균 풍속(m/s)', '경기력 일관성')
y_range_humid = weather_and_hits(df_Changwon_reset_idx, '평균 이슬점온도(°C)', '경기력 일관성')

print(y_range_temp)
print(y_range_wind)
print(y_range_humid)


[3, 4, 8, 10, 7]
[10, 10, 10, 1, 0]
[0, 5, 5, 7, 15]


In [68]:
# 3. 날씨(기온, 풍속, 이슬점) 데이터별 x, y 그래프 상 기울기의 평균 구하기
def slope(data, y_range, weather_column):
    max_val, min_val = max(data[weather_column]), min(data[weather_column])
    width = (max_val - min_val) / 5
    slope_list = []

    for j in range(len(y_range)-1):
        slope_value = (y_range[j + 1] - y_range[j]) / width
        slope_list.append(slope_value)
    slope_weather = sum(slope_list) / len(slope_list)
    return slope_weather

slope_temp = slope(df_Changwon_reset_idx, y_range_temp, '평균기온(°C)')
slope_wind = slope(df_Changwon_reset_idx, y_range_wind, '평균 풍속(m/s)')
slope_humid = slope(df_Changwon_reset_idx, y_range_humid, '평균 이슬점온도(°C)')

print(slope_temp)
print(slope_wind)
print(slope_humid)

0.25380710659898476
-5.9523809523809526
0.5514705882352942


In [69]:
# 4. 구장별 날씨(기온, 풍속, 이슬점) 데이터별 평균값 구하기
def ballpark_weather_avg(data, weather_column):
    initial_val =0

    for val in range(len(data[weather_column])):
        initial_val += data[weather_column][val]
    final_val = initial_val/len(data[weather_column])
    return final_val

ballpark_temp_avg = ballpark_weather_avg(df_Changwon_reset_idx, '평균기온(°C)')
ballpark_wind_avg = ballpark_weather_avg(df_Changwon_reset_idx, '평균 풍속(m/s)')
ballpark_humid_avg = ballpark_weather_avg(df_Changwon_reset_idx, '평균 이슬점온도(°C)')

print(ballpark_temp_avg)
print(ballpark_wind_avg)
print(ballpark_humid_avg)

20.709090909090907
1.4878787878787878
13.28787878787879


In [70]:
# 5. 구장별 파크팩터 구하기
# 5-1) 파크팩터 파일 불러오기
parkfactor_file_path ='./구장별 파크팩터_논문참고.xlsx'
df_parkfactor = pd.read_excel(parkfactor_file_path)
df_parkfactor


# 5-2) 파크팩터 중 창원의 2루타, 3루타, 홈런의 평균을 구하고 1000으로 나눠주기
df_parkfactor_changwon = df_parkfactor.loc[(df_parkfactor['구장'] == '창원'), ['2루타', '3루타', '홈런']]
df_parkfactor_changwon_avg = df_parkfactor_changwon.mean(axis=1).iloc[0] / 1000


In [71]:
# 5. 선수의 구장별 환경 점수
# 환경 점수 = (3. 기온 기울기) x (4. 구장별 기온 평균값) + (3. 풍속 기울기) x (4. 구장별 풍속 평균값) + 
#           (3. 이슬점 기울기) x (4. 구장별 이슬점 평균값) + 구장별 파크팩터/1000

player_stat = ((slope_temp * ballpark_temp_avg) + (slope_wind * ballpark_wind_avg) + \
            (slope_humid * ballpark_humid_avg) + df_parkfactor_changwon_avg).round(4)
player_stat

4.8042

결과값 정리
1. 권희동의 환경점수: -0.317 (150경기)
2. 나성범의 환경점수: 3.3258 (156경기)
3. 노진혁의 환경점수: 1.6703 (176경기)
4. 양의지의 환경점수: 8.7981 (197경기)
5. 박석민의 환경점수: 4.8042(152경기)